In [1]:
import spacy 
from spacy.tokens import DocBin
from spacy import displacy
import pandas as pd
import os

In [2]:
spacy.__version__

'3.3.1'

## Loading and evaluating the model 

In [3]:
nlp = spacy.load('models_v3.3/model-best')

In [4]:
nlp.pipe_names

['tok2vec', 'tagger', 'morphologizer', 'parser', 'trainable_lemmatizer']

In [5]:
analysis = nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component              Assigns               Requires   Scores             Retokenizes
-   --------------------   -------------------   --------   ----------------   -----------
0   tok2vec                doc.tensor                                          False      
                                                                                          
1   tagger                 token.tag                        tag_acc            False      
                                                                                          
2   morphologizer          token.morph                      pos_acc            False      
                           token.pos                        morph_acc                     
                                                            morph_per_feat                
                                                                                          
3   parser

In [7]:
# include_static_vectors = true
!python -m spacy evaluate ./models_v3.3/model-best corpus/test_udep.spacy --code language_components/custom_bg_lang.py

[i] Using CPU

================================== Results ==================================

TOK      99.97
TAG      94.75
POS      98.12
MORPH    95.79
LEMMA    93.88
UAS      89.95
LAS      84.77
SENT P   92.62
SENT R   96.77
SENT F   94.65
SPEED    984  


============================== MORPH (per feat) ==============================

                P        R        F
Case        97.11    97.21    97.16
Number      98.48    98.41    98.45
Person      97.65    97.40    97.52
PronType    98.69    98.18    98.43
Aspect      94.54    92.86    93.69
Mood        98.10    97.85    97.98
Tense       95.00    94.31    94.66
VerbForm    98.24    96.49    97.36
Voice       98.12    96.44    97.27
Definite    98.19    98.30    98.24
Degree      96.71    97.38    97.04
Gender      96.44    96.41    96.42
Poss        94.18    93.68    93.93
Reflex      98.94    99.20    99.07
Polarity   100.00   100.00   100.00
Animacy     83.33   100.00    90.91
NumType     94.88    95.14    95.01


=========

# Generating predictions 

### Packaging the model

In [8]:
!python -m spacy package ./models_v3.3/model-best/ packages --name bg --version 1.0.0 --code language_components/custom_bg_lang.py --force

running sdist
running egg_info
writing bg_bg.egg-info\PKG-INFO
writing dependency_links to bg_bg.egg-info\dependency_links.txt
writing entry points to bg_bg.egg-info\entry_points.txt
writing requirements to bg_bg.egg-info\requires.txt
writing top-level names to bg_bg.egg-info\top_level.txt
reading manifest file 'bg_bg.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'bg_bg.egg-info\SOURCES.txt'
running check
creating bg_bg-1.0.0
creating bg_bg-1.0.0\bg_bg
creating bg_bg-1.0.0\bg_bg.egg-info
creating bg_bg-1.0.0\bg_bg\bg_bg-1.0.0
creating bg_bg-1.0.0\bg_bg\bg_bg-1.0.0\morphologizer
creating bg_bg-1.0.0\bg_bg\bg_bg-1.0.0\parser
creating bg_bg-1.0.0\bg_bg\bg_bg-1.0.0\tagger
creating bg_bg-1.0.0\bg_bg\bg_bg-1.0.0\tok2vec
creating bg_bg-1.0.0\bg_bg\bg_bg-1.0.0\trainable_lemmatizer
creating bg_bg-1.0.0\bg_bg\bg_bg-1.0.0\vocab
copying files to bg_bg-1.0.0...
copying MANIFEST.in -> bg_bg-1.0.0
copying README.md -> bg_bg-1.0.0
copying meta.json -> bg_bg-1.0.0


In [9]:
!pip install packages/bg_bg-1.0.0/dist/bg_bg-1.0.0.tar.gz

Processing d:\spacy-pipeline-bg\spacy-pipeline-bg\packages\bg_bg-1.0.0\dist\bg_bg-1.0.0.tar.gz
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Created wheel for bg-bg: filename=bg_bg-1.0.0-py3-none-any.whl size=409686905 sha256=e8038ba0a824e265875364660ef44c0d5a447af3cfa19f0891ace3794755e040
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\07\e8\2c\a7e4c27bd8be30ba2969b4984d3b12b17334d8849fafa3232d
Successfully built bg-bg
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.3.0
    Uninstalling smart-open-6.3.0:
      Successfully uninstalled smart-open-6.3.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires scipy>=1.7.0, which is not installed.


### Adding the custom tokenizer

In [17]:
import spacy
nlp = spacy.load("bg_bg")

In [18]:
from language_components.custom_tokenizer import * 

In [19]:
nlp.tokenizer = custom_tokenizer(nlp)

In [20]:
nlp.pipe_names

['tok2vec', 'tagger', 'morphologizer', 'parser', 'trainable_lemmatizer']

In [21]:
# for test
def get_token_list(text): 
    doc = nlp(text)
    tokens = [t.text for t in doc]
    return tokens

In [22]:
get_token_list("Св. св. Кирил и Методий") 

['Св.', 'св.', 'Кирил', 'и', 'Методий']

# Parsing text in the Conll format 

In [23]:
text = """Честита Нова година!
Още една година се изтърколи. За едни по-добра, за други по-лоша. Без значение обаче дали 2017 година за вас е била по-динамична или по-лежерна, по-рискована или по-предвидима, по-напрегната или по-спокойна, и тя, както всички останали години, ви е донесла по нещо. Научила ви е на нещо, показала ви е нещо, помогнала ви е да израснете, поне мъничко.
И 2018 година ще е такава. Можем само да гадаем дали светът ще стане по-добър или по-лош. Можем да спекулираме за бъдещето на България и изходът от изпитанията, които я очакват. Всяка година се надяваме следващата да е по-добра, а това не винаги се случва.
Затова тази година можем да сменим подхода и да не пилеем напразна енергия в спекулации. Вместо това можем да приемем единственото сигурно нещо за настъпващата 2018 година - през нея е натрупаме още опит, без значение добър или лош, с който ще научим нови уроци - за себе си и за другите. Това е най-доброто, което можем да направим за себе си, за страната си и за света - да учим по-бързо уроците си.
Колкото по-скоро осъзнаем, че всеки трябва да е свободен да живее със своята истина, толкова по-скоро ще престанем да се съдим, да се месим където не трябва, да се мразим и да се опитваме отчаяно да убеждаваме, че има само един верен път за всичко.
Колкото по-скоро осъзнаем, че както океанът не е нищо друго, освен сбор от безброй капки, така и държавата не е нищо друго, освен сбор от милиони хора, толкова по-скоро ще си направим неизбежния извод, че нещата зависят от нас. Държавата ни ще е толкова развита, коректна и уредена, колкото сме ние. И само наше е правото да избираме съдбата си и да защитаваме този избор.
Колкото по-скоро осъзнаем, че имаме проблеми - страхове, комплекси, малодушие, толкова по-голям е шансът ни да ги припознаем като нещо, което трябва да бъде преборено, и което да НЕ предадем на децата си, за да могат те да бъдат по-добри от нас.
Уроците за всеки са различни. И никога не се учат леко. След всички трудности обаче човек излиза преобразен - малко по-спокоен, малко по-мъдър, малко по-смирен. И знае, че това е цената да бъде по-добър.
При народите е същото. И тази година ще имаме да учим общи уроци, които ще ни помогнат, ако сме достатъчно схватливи и зрели, да тръгнем напред. Ако ли не, ще почакаме. Всичко с времето си.
Честита Нова година!"""

In [24]:
!pip install spacy-conll
from spacy_conll import init_parser
nlp = init_parser("bg_bg", "spacy", include_headers=True, disable_sbd=False)

In [25]:
nlp.tokenizer = custom_tokenizer(nlp)

In [26]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'morphologizer',
 'parser',
 'trainable_lemmatizer',
 'conll_formatter']

In [27]:
conll_pd = nlp(text)._.conll_pd

In [28]:
conll_pd.head(20)

,id,form,lemma,upostag,xpostag,feats,head,deprel,deps,misc
0,1,Честита,честита,VERB,Vpitf-r3s,Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense...,3,nsubj,_,_
1,2,Нова,нов,ADJ,Afsi,Definite=Ind|Degree=Pos|Gender=Fem|Number=Sing,3,amod,_,_
2,3,година,година,NOUN,Ncfsi,Definite=Ind|Gender=Fem|Number=Sing,0,ROOT,_,SpaceAfter=No
3,4,!,!,PUNCT,punct,_,3,punct,_,SpaceAfter=No
4,1,\n,\n,PROPN,Npfsi,Definite=Ind|Gender=Fem|Number=Sing,0,ROOT,_,SpaceAfter=No
5,1,Още,още,ADV,Dq,Degree=Pos,2,advmod,_,_
6,2,една,един,NUM,Mcfsi,Definite=Ind|Gender=Fem|NumType=Card|Number=Sing,3,nummod,_,_
7,3,година,година,NOUN,Ncfsi,Definite=Ind|Gender=Fem|Number=Sing,5,nsubj,_,_
8,4,се,се,PRON,Ppxta,Case=Acc|PronType=Prs|Reflex=Yes,5,expl,_,_
9,5,изтърколи,изтърколя-(се),VERB,Vpitf-r3s,Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Tens...,0,ROOT,_,SpaceAfter=No
